In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False

import numpy as np
import pandas as pd
import time

In [2]:
df1 = pd.read_csv('./data/NSeries_Today100.txt', index_col=0)
df2 = pd.read_csv('./data/NSeries_Week100.txt', index_col=0)
df3 = pd.read_csv('./data/NSeries_Month100.txt', index_col=0)
df4 = pd.read_csv('./data/NSeries_Now100.txt', index_col=0)

In [3]:
df = pd.concat([df1, df2, df3, df4], axis=0, ignore_index=1)
df.index = df.index+1
df

,제목,장르,작가,연재 시작일,마지막 연재일,회수,댓글수,추천수,평점
1,화산귀환 [독점],무협,비가,(2019.04.25.),(2024.06.21.),1737,"1,279,634","18,135",9.2
2,등선하는 무공 천재 [독점],무협,무동이,(2024.05.07.),(2024.06.25.),112,541,8,9.6
3,농어촌 에이스 [독점],현판,이블라인,(2024.05.07.),(2024.06.24.),102,"2,802",42,9.6
4,대기업 말단이 일을 잘함 [독점],현판,동면거북이,(2023.06.13.),(2024.06.24.),371,"8,192",101,9.7
5,"꿇어보세요, 대공님! [독점]",로판,재겸,(2024.06.24.),(2024.06.25.),86,115,13,9.7
...,...,...,...,...,...,...,...,...,...
396,"나를 혐오하세요, 공작님 [독점]",로판,이초원,(2024.04.03.),(2024.06.25.),137,58,6,8.9
397,본능의 대가 [독점],로맨스,설우희,(2024.04.11.),(2024.06.17.),87,"2,040",38,9.6
398,재벌집 만렙 아들,현판,오소록,(2023.02.03.),(2024.06.24.),436,"7,426",70,9.6
399,인형의 밤 [독점],로맨스,김나결,(2023.08.04.),(2023.09.20.),90,174,25,9.0


In [4]:
# 19금 제거
index=df.loc[df.제목 == '19금'].index
index
df.drop(index, inplace=True)


In [5]:
# 제목에서 선독점 같은 상관없는 자료 제거
df.제목 = df.제목.str.replace('[단행본]', '').str.strip()
df.제목 = df.제목.str.replace('[독점]', '').str.strip()
df.제목 = df.제목.str.replace('선공개', '').str.strip()
df.제목 = df.제목.str.replace(r'\[|\]', '').str.strip()

In [6]:
# 제목이 중복되는 자료 제거
df=df.drop_duplicates('제목', keep='first', ignore_index=True)
df.제목.value_counts() # 중복값 확인

화산귀환                 1
딸기밭                  1
서부로 간 검은머리 용병        1
문제적 왕자님              1
이섭의 연애               1
                    ..
버림받고 즐기는 소박한 신의 삶    1
게임 속 검술천재가 되었다       1
사랑하는 나의 억압자          1
1500억을 받았다           1
인형의 밤                1
Name: 제목, Length: 227, dtype: int64

In [7]:
df['연재 시작일'] = df['연재 시작일'].str.replace(r'[()]', '')
df['마지막 연재일'] = df['마지막 연재일'].str.replace(r'[()]', '')

df['연재 시작일'] = pd.to_datetime(df['연재 시작일'])
df['마지막 연재일'] = pd.to_datetime(df['마지막 연재일'])


#df.회수 = df.회수.str.replace(',', '')
df.회수 = df.회수.astype(int)
df.댓글수 = df.댓글수.str.replace(',', '')
df.댓글수 = df.댓글수.astype(int)
df.추천수 = df.추천수.str.replace(',', '')
df.loc[df.추천수 == '좋아요', '추천수'] = 0
df.추천수 = df.추천수.astype(int)
df.추천수 = df.추천수.astype(int)
#df.평점 = df.평점.str.replace(',', '')
df.평점 = df.평점.astype(float)


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   제목       227 non-null    object        
 1   장르       227 non-null    object        
 2   작가       227 non-null    object        
 3   연재 시작일   227 non-null    datetime64[ns]
 4   마지막 연재일  227 non-null    datetime64[ns]
 5   회수       227 non-null    int32         
 6   댓글수      227 non-null    int32         
 7   추천수      227 non-null    int32         
 8   평점       227 non-null    float64       
dtypes: datetime64[ns](2), float64(1), int32(3), object(3)
memory usage: 13.4+ KB


In [9]:
df.장르 = df.장르.str.replace('라이트노벨', '판타지')
df.장르 = df.장르.str.replace('BL', '로맨스')

In [10]:
df.장르.value_counts()


로판     56
현판     51
로맨스    44
판타지    43
무협     33
Name: 장르, dtype: int64

In [11]:
# 나중에 쓰기 쉽게 장르 정리 true, false

all_genres = pd.Series(df['장르'].values.ravel('K')).dropna().unique()
#print(all_genres)
for genre in all_genres:
    df[genre] = df['장르'] == genre
df

,제목,장르,작가,연재 시작일,마지막 연재일,회수,댓글수,추천수,평점,무협,현판,로판,판타지,로맨스
0,화산귀환,무협,비가,2019-04-25,2024-06-21,1737,1279634,18135,9.2,True,False,False,False,False
1,등선하는 무공 천재,무협,무동이,2024-05-07,2024-06-25,112,541,8,9.6,True,False,False,False,False
2,농어촌 에이스,현판,이블라인,2024-05-07,2024-06-24,102,2802,42,9.6,False,True,False,False,False
3,대기업 말이 일을 잘함,현판,동면거북이,2023-06-13,2024-06-24,371,8192,101,9.7,False,True,False,False,False
4,"꿇어보세요, 대공님!",로판,재겸,2024-06-24,2024-06-25,86,115,13,9.7,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,투신귀환 무료연재,무협,북창,2024-05-21,2024-06-25,45,61,2,8.4,True,False,False,False,False
223,마른 가지에 바람처럼,로판,달새울,2017-09-29,2021-02-19,230,20512,4348,9.7,False,False,True,False,False
224,혼자 다 해 먹는 천재 암살자,판타지,MOZO,2022-10-12,2024-06-25,542,7227,89,9.7,False,False,False,True,False
225,"나를 혐오하세요, 공작님",로판,이초원,2024-04-03,2024-06-25,137,58,6,8.9,False,False,True,False,False


In [12]:
df.to_csv('./data/N_Best.txt')